# V9c 심층 진단 및 Breakthrough 기회 탐색
# 목표: Recall@10 15.78% → 20%+ 돌파구 찾기

In [ ]:
import os
import pickle
import random
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("V9c 진단 시작")

In [ ]:
# 설정
CONFIG = {
    'device': 'mps' if torch.backends.mps.is_available() else 'cpu',
    'processed_dir': '../data/processed',
    'model_dir': '../models',
}

# V9c CONFIG
V9C_CONFIG = {
    'embedding_dim': 64,
    'n_layers': 2,
    'temperature': 0.2,
    'neg_ratio': 6,
}

print(f"Device: {CONFIG['device']}")

## 1. 데이터 및 모델 로드

In [ ]:
# 데이터 로드
train_df = pd.read_csv(os.path.join(CONFIG['processed_dir'], 'train_split_v6.csv'))
valid_df = pd.read_csv(os.path.join(CONFIG['processed_dir'], 'valid_split_v6.csv'))
test_df = pd.read_csv(os.path.join(CONFIG['processed_dir'], 'test_split_v6.csv'))

with open(os.path.join(CONFIG['processed_dir'], 'id_mappings_v6.pkl'), 'rb') as f:
    mappings = pickle.load(f)

n_users = len(mappings['user_id_map'])
n_items = len(mappings['item_id_map'])

print(f"Users: {n_users}, Items: {n_items}")
print(f"Train: {len(train_df):,}, Valid: {len(valid_df):,}, Test: {len(test_df):,}")

In [ ]:
# LightGCN 모델 정의 (V9c와 동일)
class LightGCNConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr='add')
    
    def forward(self, x, edge_index):
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        return self.propagate(edge_index, x=x, norm=norm)
    
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

class LightGCN(nn.Module):
    def __init__(self, n_users, n_items, embedding_dim=64, n_layers=2):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers
        
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        nn.init.normal_(self.user_embedding.weight, std=0.1)
        nn.init.normal_(self.item_embedding.weight, std=0.1)
        
        self.convs = nn.ModuleList([LightGCNConv() for _ in range(n_layers)])
    
    def forward(self, edge_index):
        user_emb = self.user_embedding.weight
        item_emb = self.item_embedding.weight
        all_emb = torch.cat([user_emb, item_emb], dim=0)
        
        embs = [all_emb]
        for conv in self.convs:
            all_emb = conv(all_emb, edge_index)
            embs.append(all_emb)
        
        final_emb = torch.stack(embs, dim=0).mean(dim=0)
        user_final = final_emb[:self.n_users]
        item_final = final_emb[self.n_users:]
        
        return user_final, item_emb

# V9c 모델 로드
model = LightGCN(n_users, n_items, V9C_CONFIG['embedding_dim'], V9C_CONFIG['n_layers'])
model.load_state_dict(torch.load(os.path.join(CONFIG['model_dir'], 'lightgcn_v9c_best.pth')))
model = model.to(CONFIG['device'])
model.eval()

print("✅ V9c 모델 로드 완료")

In [ ]:
# Graph 생성
def create_graph(df, n_users, n_items):
    user_ids = df['user_id'].values
    item_ids = df['item_id'].values + n_users
    edge_index = torch.tensor([
        np.concatenate([user_ids, item_ids]),
        np.concatenate([item_ids, user_ids])
    ], dtype=torch.long)
    return edge_index

train_valid_df = pd.concat([train_df, valid_df])
edge_index = create_graph(train_valid_df, n_users, n_items).to(CONFIG['device'])

print(f"Graph edges: {edge_index.shape[1]:,}")

## 2. 예측 품질 분석 - 어디서 실패하나?

In [ ]:
# Test set에서 user별 성능 분석
print("=" * 60)
print("User별 예측 품질 분석")
print("=" * 60)

with torch.no_grad():
    user_emb, item_emb = model(edge_index)

# User-item dictionary
train_valid_items = defaultdict(set)
for _, row in train_valid_df.iterrows():
    train_valid_items[row['user_id']].add(row['item_id'])

user_recalls = []
user_precisions = []
user_test_counts = []
user_train_counts = []

for user_id, group in test_df.groupby('user_id'):
    true_items = set(group['item_id'].values)
    exclude_items = train_valid_items[user_id]
    
    # 예측
    user_emb_single = user_emb[user_id].unsqueeze(0)
    scores = torch.matmul(user_emb_single, item_emb.t()).squeeze()
    scores_np = scores.cpu().numpy()
    
    for item_id in exclude_items:
        scores_np[int(item_id)] = -np.inf
    
    top_10 = np.argsort(scores_np)[-10:][::-1]
    hits = len(set(top_10) & true_items)
    
    recall = hits / len(true_items) if len(true_items) > 0 else 0
    precision = hits / 10
    
    user_recalls.append(recall)
    user_precisions.append(precision)
    user_test_counts.append(len(true_items))
    user_train_counts.append(len(exclude_items))

user_recalls = np.array(user_recalls)
user_precisions = np.array(user_precisions)
user_test_counts = np.array(user_test_counts)
user_train_counts = np.array(user_train_counts)

print(f"\n전체 평균 Recall@10: {user_recalls.mean():.4f}")
print(f"Recall=0인 user 비율: {(user_recalls == 0).sum() / len(user_recalls):.2%}")
print(f"Recall=1인 user 비율: {(user_recalls == 1).sum() / len(user_recalls):.2%}")

In [ ]:
# Recall 분포 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Recall 분포
axes[0, 0].hist(user_recalls, bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(user_recalls.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean={user_recalls.mean():.3f}')
axes[0, 0].set_xlabel('Recall@10')
axes[0, 0].set_ylabel('User Count')
axes[0, 0].set_title('User별 Recall@10 분포')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Train size vs Recall
axes[0, 1].scatter(user_train_counts, user_recalls, alpha=0.5, s=30)
axes[0, 1].set_xlabel('Train Interaction Count')
axes[0, 1].set_ylabel('Recall@10')
axes[0, 1].set_title('Train 데이터 크기 vs 성능')
axes[0, 1].grid(True, alpha=0.3)

# Test size vs Recall
axes[1, 0].scatter(user_test_counts, user_recalls, alpha=0.5, s=30, color='orange')
axes[1, 0].set_xlabel('Test Item Count')
axes[1, 0].set_ylabel('Recall@10')
axes[1, 0].set_title('Test 아이템 수 vs 성능 (많을수록 어려움)')
axes[1, 0].grid(True, alpha=0.3)

# Recall 분위별 통계
quantiles = [0, 0.25, 0.5, 0.75, 1.0]
quantile_vals = np.quantile(user_recalls, quantiles)
axes[1, 1].bar(range(len(quantiles)), quantile_vals, tick_label=['0%', '25%', '50%', '75%', '100%'])
axes[1, 1].set_xlabel('Percentile')
axes[1, 1].set_ylabel('Recall@10')
axes[1, 1].set_title('Recall 분위수')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../results/v9c_user_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ User별 분석 완료")

In [ ]:
# 성능이 좋은 user vs 나쁜 user 비교
print("=" * 60)
print("High vs Low Performance User 비교")
print("=" * 60)

high_recall_users = user_recalls > np.percentile(user_recalls, 75)
low_recall_users = user_recalls < np.percentile(user_recalls, 25)

print(f"\nHigh Performance Users (top 25%):")
print(f"  평균 Recall: {user_recalls[high_recall_users].mean():.4f}")
print(f"  평균 Train size: {user_train_counts[high_recall_users].mean():.1f}")
print(f"  평균 Test size: {user_test_counts[high_recall_users].mean():.1f}")

print(f"\nLow Performance Users (bottom 25%):")
print(f"  평균 Recall: {user_recalls[low_recall_users].mean():.4f}")
print(f"  평균 Train size: {user_train_counts[low_recall_users].mean():.1f}")
print(f"  평균 Test size: {user_test_counts[low_recall_users].mean():.1f}")

print(f"\n🔍 핵심 발견:")
if user_test_counts[low_recall_users].mean() > user_test_counts[high_recall_users].mean():
    print("  ⚠️  Low performance user들은 test item이 더 많음 → 더 어려운 task")
if user_train_counts[low_recall_users].mean() < user_train_counts[high_recall_users].mean():
    print("  ⚠️  Low performance user들은 train data가 적음 → cold-start 문제")

## 3. Score Distribution 분석

In [ ]:
# Score distribution
print("=" * 60)
print("Score Distribution 분석")
print("=" * 60)

sample_user = 0
user_emb_single = user_emb[sample_user].unsqueeze(0)
all_scores = torch.matmul(user_emb_single, item_emb.t()).squeeze().cpu().numpy()

print(f"\nScore 통계 (User {sample_user}):")
print(f"  Mean: {all_scores.mean():.4f}")
print(f"  Std: {all_scores.std():.4f}")
print(f"  Min: {all_scores.min():.4f}")
print(f"  Max: {all_scores.max():.4f}")

# 전체 user 평균
all_user_scores_mean = []
all_user_scores_std = []

for uid in range(min(100, n_users)):  # 샘플링
    scores = torch.matmul(user_emb[uid].unsqueeze(0), item_emb.t()).squeeze().cpu().numpy()
    all_user_scores_mean.append(scores.mean())
    all_user_scores_std.append(scores.std())

print(f"\n전체 User 평균 (100명 샘플):")
print(f"  Score mean의 평균: {np.mean(all_user_scores_mean):.4f}")
print(f"  Score std의 평균: {np.mean(all_user_scores_std):.4f}")

if np.mean(all_user_scores_std) < 0.1:
    print("\n⚠️  경고: Score diversity가 낮음! → Score collapse 가능성")
else:
    print("\n✅ Score diversity는 건강함")

## 4. Embedding Quality 분석

In [ ]:
# Embedding similarity 분석
print("=" * 60)
print("Embedding Quality 분석")
print("=" * 60)

# Item embedding similarity
item_emb_np = item_emb.cpu().numpy()
item_emb_normalized = item_emb_np / (np.linalg.norm(item_emb_np, axis=1, keepdims=True) + 1e-8)
item_similarity_matrix = np.dot(item_emb_normalized, item_emb_normalized.T)

# 대각선 제외하고 평균 similarity
mask = ~np.eye(item_similarity_matrix.shape[0], dtype=bool)
avg_item_similarity = item_similarity_matrix[mask].mean()

print(f"\nItem Embedding Similarity:")
print(f"  평균 cosine similarity: {avg_item_similarity:.4f}")

if avg_item_similarity > 0.5:
    print("  ⚠️  경고: Item embedding들이 너무 유사함! → Embedding collapse")
elif avg_item_similarity < 0.1:
    print("  ✅ Item embedding이 다양함 (건강)")
else:
    print("  ✅ 적절한 수준의 similarity")

# Embedding norm 분포
item_norms = np.linalg.norm(item_emb_np, axis=1)
print(f"\nItem Embedding Norm:")
print(f"  Mean: {item_norms.mean():.4f}")
print(f"  Std: {item_norms.std():.4f}")
print(f"  Min: {item_norms.min():.4f}")
print(f"  Max: {item_norms.max():.4f}")

## 5. Negative Sample Quality 분석

In [ ]:
# Random negative vs Hard negative 비교
print("=" * 60)
print("Negative Sample Quality 분석")
print("=" * 60)

sample_user = 0
user_emb_single = user_emb[sample_user].unsqueeze(0)
all_scores = torch.matmul(user_emb_single, item_emb.t()).squeeze().cpu().numpy()

# Positive items (train)
pos_items = list(train_valid_items[sample_user])
if pos_items:
    pos_scores = all_scores[pos_items]
    print(f"\nPositive Item Scores (User {sample_user}):")
    print(f"  Mean: {pos_scores.mean():.4f}")
    print(f"  Std: {pos_scores.std():.4f}")

# Random negative sampling
all_items = set(range(n_items))
neg_candidates = list(all_items - train_valid_items[sample_user])
random_negs = np.random.choice(neg_candidates, size=100, replace=False)
random_neg_scores = all_scores[random_negs]

print(f"\nRandom Negative Scores:")
print(f"  Mean: {random_neg_scores.mean():.4f}")
print(f"  Std: {random_neg_scores.std():.4f}")

# Hard negatives (high score non-positives)
neg_scores_sorted = sorted([(i, all_scores[i]) for i in neg_candidates], key=lambda x: x[1], reverse=True)
hard_negs = [x[0] for x in neg_scores_sorted[:100]]
hard_neg_scores = all_scores[hard_negs]

print(f"\nHard Negative Scores (top 100 non-positives):")
print(f"  Mean: {hard_neg_scores.mean():.4f}")
print(f"  Std: {hard_neg_scores.std():.4f}")

print(f"\n🔍 분석:")
if pos_items:
    pos_mean = pos_scores.mean()
    rand_mean = random_neg_scores.mean()
    hard_mean = hard_neg_scores.mean()
    
    print(f"  Positive vs Random gap: {pos_mean - rand_mean:.4f}")
    print(f"  Positive vs Hard gap: {pos_mean - hard_mean:.4f}")
    
    if pos_mean - hard_mean < 0.1:
        print("  ⚠️  Hard negative가 positive와 너무 가까움!")
        print("  → Hard negative mining이 학습에 큰 도움이 될 것")
    
    if pos_mean - rand_mean > 1.0:
        print("  ⚠️  Random negative가 너무 쉬움!")
        print("  → 모델이 이미 쉬운 negative는 잘 구분함")
        print("  → Hard negative mining 필요!")

## 6. 데이터 특성 재검토

In [ ]:
# 데이터 분포 재확인
print("=" * 60)
print("데이터 특성 재검토")
print("=" * 60)

# Item degree 분포
item_degrees = train_valid_df['item_id'].value_counts().sort_index()
print(f"\nItem Degree 통계:")
print(f"  Mean: {item_degrees.mean():.2f}")
print(f"  Median: {item_degrees.median():.2f}")
print(f"  Min: {item_degrees.min()}")
print(f"  Max: {item_degrees.max()}")
print(f"  Degree < 10: {(item_degrees < 10).sum()} items ({(item_degrees < 10).sum()/len(item_degrees):.1%})")
print(f"  Degree < 20: {(item_degrees < 20).sum()} items ({(item_degrees < 20).sum()/len(item_degrees):.1%})")

# User degree 분포
user_degrees = train_valid_df['user_id'].value_counts().sort_index()
print(f"\nUser Degree 통계:")
print(f"  Mean: {user_degrees.mean():.2f}")
print(f"  Median: {user_degrees.median():.2f}")
print(f"  Min: {user_degrees.min()}")
print(f"  Max: {user_degrees.max()}")

# Sparsity
sparsity = 1 - len(train_valid_df) / (n_users * n_items)
print(f"\nSparsity: {sparsity:.4f} ({sparsity*100:.2f}%)")

if (item_degrees < 20).sum() / len(item_degrees) > 0.5:
    print("\n⚠️  50% 이상의 item이 degree < 20")
    print("  → 데이터가 여전히 sparse함")
    print("  → 더 aggressive한 filtering 고려 가능")

## 7. Breakthrough 기회 요약

In [ ]:
print("=" * 60)
print("🎯 BREAKTHROUGH 기회 요약")
print("=" * 60)

print("\n진단 결과 기반 우선순위:")
print("\n1. 🔥 Hard Negative Mining (최우선)")
print("   - Random negative가 너무 쉬움")
print("   - Hard negative와 positive 구분이 핵심")
print("   - 예상 개선: +4-8% Recall@10")
print("   - 구현: Semi-hard negative, In-batch negative")

print("\n2. 💪 Embedding Capacity 증가")
print("   - 현재 64-dim → 128 또는 256으로")
print("   - Params/data ratio: 3.2x (여전히 안전)")
print("   - 예상 개선: +2-4% Recall@10")
print("   - 리스크: Low (overfitting 가능성 낮음)")

print("\n3. 🔄 Learning Rate Scheduling")
print("   - 현재: Fixed LR 0.001")
print("   - 개선: Cosine annealing with warm-up")
print("   - 예상 개선: +1-2% Recall@10")
print("   - 리스크: Low")

print("\n4. 📊 더 Aggressive한 Data Filtering")
print("   - 현재: min_user=30, min_item=10")
print("   - 개선: min_item=15 or 20")
print("   - Trade-off: 데이터 감소 vs 품질 향상")
print("   - 예상: 불확실 (실험 필요)")

print("\n5. 🎨 Graph Augmentation")
print("   - Edge dropout during training")
print("   - Node feature noise")
print("   - 예상 개선: +1-3% Recall@10")
print("   - 리스크: Medium (구현 복잡도)")

print("\n6. 🔗 Ensemble")
print("   - V6 (BPR) + V9c (InfoNCE)")
print("   - 예상 개선: +1-2% Recall@10")
print("   - 단점: 추론 시간 2배")

print("\n" + "=" * 60)
print("💡 제안 실행 계획:")
print("=" * 60)
print("\nPhase 1 (병렬 실행 가능):")
print("  V10a: Embedding 128 (2-3시간)")
print("  V11a: Hard negative mining (3-4시간)")
print("  → 목표: 18-20% Recall@10")

print("\nPhase 2 (Phase 1 결과 기반):")
print("  V12: Best from Phase 1 + LR scheduling")
print("  V13: Graph augmentation")
print("  → 목표: 20-22% Recall@10")

print("\nPhase 3 (최종 푸시):")
print("  V14: Ensemble of best models")
print("  → 목표: 22-25% Recall@10")

print("\n✅ 진단 완료!")